## 2. Affine camera calibration

In [55]:
"""CS231A Homework 1, Problem 2.

DATA FORMAT
In this problem, we provide and load the data for you. Recall that in the
original problem statement, there exists a grid of black squares on a white
background. We know how these black squares are setup, and thus can determine
the locations of specific points on the grid (namely the corners). We also have
images taken of the grid at a front image (where Z = 0) and a back image (where
Z = 150). The data we load for you consists of three parts: real_XY,
front_image, and back_image. For a corner (0,0), we may see it at the (137, 44)
pixel in the front image and the (148, 22) pixel in the back image. Thus, one
row of real_XY will contain the numpy array [0, 0], corresponding to the real
XY location (0, 0). The matching row in front_image will contain [137, 44] and
the matching row in back_image will contain [148, 22].
"""

import numpy as np
import math

def extend_real_coordinates(coords, z):        
    z_extend = z * np.ones((real_XY.shape[0], 1))
    hg_extend = np.ones((real_XY.shape[0], 1))
    zeros = np.zeros((real_XY.shape[0], 4))
    coords_x = np.concatenate((coords, z_extend, hg_extend, zeros), axis=1)
    coords_y = np.concatenate((zeros, coords, z_extend, hg_extend), axis=1)
    return np.concatenate((coords_x, coords_y), axis=0)


def compute_camera_matrix(real_XY, front_image, back_image):
    """Computes camera matrix given image and real-world coordinates.

    Args:
        real_XY: Each row corresponds to an actual point on the 2D plane.
        front_image: Each row is the pixel location in the front image (Z=0).
        back_image: Each row is the pixel location in the back image (Z=150).
    Returns:
        camera_matrix: The calibrated camera matrix (3x4 matrix).
    """
    
    real_XY_front = extend_real_coordinates(real_XY, 0)
    real_XY_back = extend_real_coordinates(real_XY, 150)
    real_XY_complete = np.concatenate((real_XY_front, real_XY_back), axis=0)
    
    image_XY_complete = np.concatenate(
        (front_image.flatten(order='F'), back_image.flatten(order='F')))
    
    lstsq_solution = np.linalg.lstsq(real_XY_complete, image_XY_complete)
    camera_matrix = np.concatenate(
        (np.reshape(lstsq_solution[0], (2, 4)), np.array([[0, 0, 0, 1]])), axis=0)
    
    return camera_matrix

def rms_error(camera_matrix, real_XY, front_image, back_image):
    """Computes RMS error of points reprojected into the images.

    Args:
        camera_matrix: The camera matrix of the calibrated camera.
        real_XY: Each row corresponds to an actual point on the 2D plane.
        front_image: Each row is the pixel location in the front image (Z=0).
        back_image: Each row is the pixel location in the back image (Z=150).
    Returns:
        rms_error: The root mean square error of reprojecting the points back
            into the images.
    """
    real_XY_front = extend_real_coordinates(real_XY, 0)
    real_XY_back = extend_real_coordinates(real_XY, 150)
    real_XY_complete = np.concatenate((real_XY_front, real_XY_back), axis=0)
    
    image_XY_complete = np.concatenate(
        (front_image.flatten(order='F'), back_image.flatten(order='F')))
    
    camera_params = np.concatenate((camera_matrix[0, :], camera_matrix[1, :]), axis=0)
    calculated_coords = np.dot(real_XY_complete, camera_params)
    diff = calculated_coords - image_XY_complete
    rms = math.sqrt(np.dot(diff, diff) / diff.shape[0])    
    return rms
    

# Load the example coordinates setup.
real_XY = np.load('real_XY.npy')
front_image = np.load('front_image.npy')
back_image = np.load('back_image.npy')

camera_matrix = compute_camera_matrix(real_XY, front_image, back_image)
rmse = rms_error(camera_matrix, real_XY, front_image, back_image)

print ("Camera Matrix: \n", camera_matrix)
print ("")
print ("RMS Error: ", rmse)



Camera Matrix: 
 [[  5.31276507e-01  -1.80886074e-02   1.20509667e-01   1.29720641e+02]
 [  4.84975447e-02   5.36366401e-01  -1.02675222e-01   4.43879607e+01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]

RMS Error:  0.70274427407708


## 3. Single View Geometry

In [47]:
"""CS231A Homework 1, Problem 3."""

import math
import numpy as np
import numpy.matlib as matlib
from utils import mat2euler


def compute_line_equation(points):
    point1 = np.concatenate((points[0], [1]))
    point2 = np.concatenate((points[1], [1]))
    result = np.cross(point1, point2)
    return result


def compute_vanishing_point(points):
    """Computes vanishing point given four points on parallel line.

    Args:
        points: A list of all the points where each row is (x, y). Generally,
            it will contain four points: two for each parallel line.
            You can use any convention you'd like, but our solution uses the
            first two rows as points on the same line and the last
            two rows as points on the same line.
    Returns:
        vanishing_point: The pixel location of the vanishing point.
    """
    line1 = compute_line_equation(points[0:2])
    line2 = compute_line_equation(points[2:4])
    vanish_point_hg = np.cross(line1, line2)
    vanish_point = vanish_point_hg[0:2] / vanish_point_hg[2]
    return vanish_point

def compute_K_from_vanishing_points(vanishing_points):
    """Compute intrinsic matrix given vanishing points.

    Args:
        vanishing_points: A list of vanishing points.
    Returns:
        K: The intrinsic camera matrix (3x3 matrix).
    """
    def compute_coefficients(v1, v2):
        return np.array([np.dot(v1, v2), v1[0] + v2[0], v1[1] + v2[1]])
    
    A = np.zeros((3, 3))
    A[0, :] = compute_coefficients(vanishing_points[0], vanishing_points[1])
    A[1, :] = compute_coefficients(vanishing_points[0], vanishing_points[2])
    A[2, :] = compute_coefficients(vanishing_points[1], vanishing_points[2])
    b = -1 * np.ones((3, 1))

    w = np.linalg.solve(A, b)
    W = np.zeros((3, 3))
    W[0, 0] = w[0]
    W[1, 1] = w[0]
    W[0, 2] = w[1]
    W[2, 0] = w[1]
    W[1, 2] = w[2]
    W[2, 1] = w[2]
    W[2, 2] = 1
    k = np.linalg.cholesky(W)
    K = np.linalg.inv(k.T)

    return (1./K[2,2]) * K
    
    
def compute_angle_between_planes(vanishing_pair1, vanishing_pair2, K):
    """Compute angle between planes of the given pairs of vanishing points.

    Args:
        vanishing_pair1: A list of a pair of vanishing points computed from
            lines within the same plane.
        vanishing_pair2: A list of another pair of vanishing points from a
            different plane than vanishing_pair1.
        K: The camera matrix used to take both images.
    Returns:
        angle: The angle in degrees between the planes which the vanishing
            point pair comes from2.
    """
    W_inv = np.dot(K, K.T)
    
    l1 = np.cross(np.append(vanishing_pair1[0], [1]), np.append(vanishing_pair1[1], [1]))
    l2 = np.cross(np.append(vanishing_pair2[0], [1]), np.append(vanishing_pair2[1], [1]))
    cos_angle = np.dot(l1, np.dot(W_inv, l2)) / (
        math.sqrt(np.dot(l1, np.dot(W_inv, l1))) * math.sqrt(np.dot(l2, np.dot(W_inv, l2))))
    return math.degrees(np.arccos(cos_angle))
    
def compute_rotation_matrix_between_cameras(vanishing_pts1, vanishing_pts2, K):
    """Compute rotation matrix between two cameras given their vanishing points.

    Args:
        vanishing_pts1: A list of vanishing points in image 1.
        vanishing_pts2: A list of vanishing points in image 2.
        K: The camera matrix used to take both images.

    Returns:
        R: The rotation matrix between camera 1 and camera 2.
    """
    K_inv = np.linalg.inv(K)
    
    def compute_dirs(vanishing_pts, K_inv):
        vanishing_pts_hg = np.concatenate((vanishing_pts, np.ones((vanishing_pts.shape[0],1))), axis=1).T    
        dirs = np.dot(K_inv, vanishing_pts_hg)
        dirs_norm = np.sqrt(np.diag(np.dot(dirs.T, dirs)))
        dirs_norm_inv = 1. / dirs_norm
        dirs_norm_inv = matlib.repmat(dirs_norm_inv, dirs.shape[0], 1)
        dirs = np.multiply(dirs, dirs_norm_inv)
        return dirs
    
    dirs_1 = compute_dirs(vanishing_pts1, K_inv)
    dirs_2 = compute_dirs(vanishing_pts2, K_inv)

    R0 = np.linalg.solve(dirs_1.T, dirs_2[0,:])    
    R1 = np.linalg.solve(dirs_1.T, dirs_2[1,:])
    R2 = np.linalg.solve(dirs_1.T, dirs_2[2,:])
    
    R = np.array([R0, R1, R2])
    return R


# Part A: Compute vanishing points.
v1 = compute_vanishing_point(np.array(
        [[674, 1826], [2456, 1060], [1094, 1340], [1774, 1086]]))
v2 = compute_vanishing_point(np.array(
        [[674, 1826], [126, 1056], [2456, 1060], [1940, 866]]))
v3 = compute_vanishing_point(np.array(
        [[1094, 1340], [1080, 598], [1774, 1086], [1840, 478]]))

v1b = compute_vanishing_point(np.array(
        [[314, 1912], [2060, 1040], [750, 1378], [1438, 1094]]))
v2b = compute_vanishing_point(np.array(
        [[314, 1912], [36, 1578], [2060, 1040], [1598, 882]]))
v3b = compute_vanishing_point(np.array(
        [[750, 1378], [714, 614], [1438, 1094], [1474, 494]]))

# Part B: Compute the camera matrix.
vanishing_points = [v1, v2, v3]
K_ours = compute_K_from_vanishing_points(vanishing_points)
print ("Intrinsic Matrix:\n", K_ours)

K_actual = np.array([[2448.0, 0, 1253.0], [0, 2438.0, 986.0], [0, 0, 1.0]])
print ("")
print ("Actual Matrix:\n", K_actual)

# Part D: Estimate the angle between the box and floor.
floor_vanishing1 = v1
floor_vanishing2 = v2
box_vanishing1 = v3
box_vanishing2 = compute_vanishing_point(np.array(
        [[1094, 1340], [1774, 1086], [1080, 598], [1840, 478]]))
angle = compute_angle_between_planes(
        [floor_vanishing1, floor_vanishing2],
        [box_vanishing1, box_vanishing2], K_actual)
print ("")
print ("Angle between floor and box:", angle)

# Part E: Compute the rotation matrix between the two cameras.
rotation_matrix = compute_rotation_matrix_between_cameras(
        np.array([v1, v2, v3]), np.array([v1b, v2b, v3b]), K_actual)
print
print ("Rotation between two cameras:\n", rotation_matrix)
z, y, x = mat2euler(rotation_matrix)
x_angle = x * 180 / math.pi
y_angle = y * 180 / math.pi
z_angle = z * 180 / math.pi
print
print ("Angle around z-axis (pointing out of camera): %f degrees" % z_angle)
print ("Angle around y-axis (pointing vertically): %f degrees" % y_angle)
print ("Angle around x-axis (pointing horizontally): %f degrees" % x_angle)



Intrinsic Matrix:
 [[  2.59416985e+03   0.00000000e+00   7.73289548e+02]
 [  0.00000000e+00   2.59416985e+03   9.79503278e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]

Actual Matrix:
 [[  2.44800000e+03   0.00000000e+00   1.25300000e+03]
 [  0.00000000e+00   2.43800000e+03   9.86000000e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]

Angle between floor and box: 90.027361241031
Rotation between two cameras:
 [[ 0.96154157  0.04924778 -0.15783349]
 [-0.01044314  1.00703585  0.04571333]
 [ 0.18940319 -0.06891607  1.00470583]]
Angle around z-axis (pointing out of camera): -2.931986 degrees
Angle around y-axis (pointing vertically): -8.918793 degrees
Angle around x-axis (pointing horizontally): -2.605117 degrees
